Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Bilal Siddiqi"
COLLABORATORS = ""

---

### <font color="brown">Data Management for Data Science</font>
### <font color="brown">Lab 04: Databases and SQL </font>
### <font color="brown">Due Date : Sunday Nov 23, 2025 11.59PM </font>
You can still submit 48 hours late. No assignment is accepted after 48 hrs. But you will only have 4 total late days for all assignments, max 2 late days per assignment. Additional late days will be penalized.

<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself.<br>
<b>Name</b>:Bilal Siddiqi<br> 
<b>NetID</b>:bms283<br>
<b>Recitation #</b>:5<br>
<b>Notes to Grader</b> (optional): hi<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://studentconduct.rutgers.edu/processes/academic-integrity">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:      BS

## The Problem
You’ve been hired as a junior data engineer for an online learning platform that sells video-based courses to students around the world. Right now, all enrollment information is kept in a single big spreadsheet. This causes:


 - Data inconsistency (student names/emails typed differently)
 - Redundancy (same course and instructor info repeated many times)
 - Difficulty in querying (hard to answer questions like “Which courses bring the most revenue?”)


You are asked to:

1. Explore a raw CSV file containing enrollment data
2. Design a normalized relational schema
3. Create SQL tables using SQLite
4. Load data from the raw CSV into a staging table
5. Populate normalized tables from the staging table
6. Practice `UPDATE`, `DELETE`, and analytical `SELECT` queries
7. Reflect on normalization and anomalies


## Use of cGPT
You are allowed to use cGPT as you complete tasks. If a code snippet was generated by cGPT, then leave a comment in the line/block stating <br>"\# generated by cGPT". Always be skeptical of AI. It is important that you understand the AI generated output 100% to avoid any errors.

## Setup

We will use **SQLite** via Python's built-in `sqlite3` module.

**Initialize:**
1. Run the cell below to create a local SQLite database file `edutrack.db`.
2. Use helper functions to execute SQL and display results.


In [82]:
import sqlite3
import pandas as pd
from pathlib import Path

db_path = Path("edutrack.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def run_sql(sql, params=None, show=True):
    """Run a SQL command (or multiple commands) and optionally show results as a DataFrame."""
    if params is None:
        params = ()
    try:
        cursor.executescript(sql) if "\n" in sql.strip().split(" ")[0] else cursor.execute(sql, params)
        conn.commit()
        if show and sql.lstrip().lower().startswith("select"):
            rows = cursor.fetchall()
            cols = [desc[0] for desc in cursor.description]
            display(pd.DataFrame(rows, columns=cols))
    except Exception as e:
        print("Error:", e)

print("Connected to:", db_path.resolve())


query = """SELECT StudentName FROM Students GROUP BY StudentName """
run_sql(query)

Connected to: /home/bms283/cs210f25/Lab 04 - Databases and SQL/edutrack.db


,StudentName
0,Bob Smith
1,Carla Gomez
2,Daniel Lee
3,Alice Johnson
4,BOB SMITH
5,Bob Smith
6,CARLA GOMEZ
7,Carla Gomez
8,DANIEL LEE
9,Daniel Lee


## Part 1 – Explore the Raw Data

The CSV data file is stored in /srv/shared/data. Read the file and create a DataFrame to inspect columns and think about redundancy and anomalies.

#### Your Task
- List the columns.
- Identify repeated information.
- Describe at least 3 potential anomalies (and update/insert/delete to fix them).


In [83]:
import pandas as pd
df_raw = pd.read_csv("/srv/shared/data/EduTrack_Enrollments_Raw.csv")
df_raw

,OrderID,OrderDate,StudentName,StudentEmail,StudentCity,CourseCode,CourseTitle,CourseCategory,InstructorName,InstructorEmail,Price,PaymentMethod,DiscountCode,EnrollmentStatus
0,1001,2024-09-03,DANIEL LEE,daniel.lee@example.com,San Francisco,DS101,Intro to Data Science,data science,Dr. Emily Zhang,emily.zhang@edutrack.com,FREE,NaN,SUMMER10,Completed
1,1001,2024-09-01,Bob Smith,bob.smith@example.com,new york,SQL201,Relational Databases for Analysts,Databases,Prof. Raj Patel,raj.patel@edutrack.com,FREE,Credit Card,SUMMER10,Actve
2,1002,2024-09-03,Daniel Lee,daniel.lee@example.com,San Francisco,DS101,intro to data science,Data Science,Dr. Emily Zhang,emily.zhang@edutrack.com,99.0,PayPal,NaN,active
3,1003,2024-09-01,Bob Smith,bob.smith@example.com,New York,SQL201,RELATIONAL DATABASES FOR ANALYSTS,Databases,Prof. Raj Patel,raj.patel@edutrack.com,79,Credit Card,SUMMER10,Refunded
4,1004,2024-09-03,Daniel Lee,daniel.lee@example.com,San Francisco,DS101,Intro to Data Science,Data Science,Dr. Emily Zhang,emily.zhang@edutrack.com,99.0,Credit Card,SUMMER10,Actve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1093,2024-09-02,Carla Gomez,carla.gomez@example.com,Miami,ML301,Intro to Machine Learning,Machine Learning,Dr Emily Zhang,emily.zhang@edutrack.com,124,credit card,NaN,completed
96,1094,2024-09-02,Carla Gomez,CARLA.GOMEZ@EXAMPLE.COM,Miami,ML301,Intro to Machine Learning,Machine Learning,Dr Emily Zhang,emily.zhang@edutrack.com,129,PayPal,NaN,Actve
97,1095,2024-09-03,DANIEL LEE,daniel.lee@example.com,San Francisco,DS101,INTRO TO DATA SCIENCE,Data Science,Dr. Emily Zhang,emily.zhang@edutrack.com,FREE,Credit Card,NaN,completed
98,1096,2024-09-02,carla gomez,carla.gomez@example.com,miami,ML301,Intro to Machine Learning,machine learning,Dr Emily Zhang,emily.zhang@edutrack.com,129.0,credit card,NaN,Completed


### Task 1.1 - Your Analysis (10 pts)
There may be many anomolies in this data set. Identify, suggest and fix. 
 - inconsistent casing
 - stray spaces
 - varied instructor name formats
 - mixed price formats (FREE, 0, integer, float, noisy ± values)
 - missing payment methods
 - inconsistent discount codes (null, None, empty)
 - messy status values
 - occasional duplicate order IDs
 - randomized corruption across fields

Write at least 3 anamolies you found and methods you will use to fix those. If there are no anomolies, state so and justify the answer.


##### Write your answer here
There are multiple anomalies which can be seen in the code output. One of which is inconsistent capitailization. This can be fixed through existing functions which can turn the entire column uppercase or lowercase. Additionally, there is inconsistent formatting in the price column, with some values having decimal points and some just being integers. Some regex could be used to fix this. Additionally, DiscountCode and PaymentMethods have null values. While it may be ok to keep empty values for the DiscountCode column, the PaymentMethods column should have no nulls since everyone has to have a payment method in order to use the website.

### Task 1.2 - Clean the file 
Using various data cleaning methods, handle anamolies. Comment each code block to show which one's are cleaned. 

In [7]:
## your code here to fix errors
df_clean = df_raw.copy()

for x in df_clean.columns: 
    if (type(df_clean[x][1]) == str):
        df_clean[x] = df_clean[x].str.lower()   #Fixes inconsistent casing by making all string cols have lowercase values.
        df_clean[x] = df_clean[x].str.strip() #Removes any excess white space.

        

df_clean = df_clean.dropna(subset=['PaymentMethod']) #Dropping columns with no payment method attached since payment method is required to use the website.


df_clean['DiscountCode'] = df_clean['DiscountCode'].fillna('no_code')    #Replaces null values for DiscountCode with a filler value since a discount code isn't neccessary.

df_clean['Price'] = df_clean['Price'].replace('free', 0)
df_clean['Price'] = df_clean['Price'].astype(float).astype(int)               #Gets rid of string values in price and sets all prices to a consistent format.

df_clean['InstructorName'] = df_clean['InstructorName'].replace('dr emily zhang', 'dr. emily zhang') #Sets instructor names to consistent format.

df_clean['EnrollmentStatus'] = df_clean['EnrollmentStatus'].replace('actve', 'active') #Replaces some of the messy status values.

df_clean['OrderID'] = df_clean['OrderID'].astype(str) + "_" + df_clean['OrderDate'].astype(str) #Makes all OrderIDs unique.


### Task 1.2 Save the Cleaned File

<b>Create a local data folder</b> and save the cleaned file <b>data/EduTrack_Enrollments_Cleaned.csv</b>. Note that the file should be stored locally in the data folder in your lab folder.

In [8]:
import os

if not os.path.exists('data'):
    os.makedirs('data')


df_clean.to_csv('data/EduTrack_Enrollments_Cleaned.csv', index=False)



## Part 2 – Design a Normalized Schema (25 pts)

In this section, you should:

1. Identify functional dependencies
2. Propose tables (name, columns, primary key, foreign keys)
3. (Optionally) sketch an ER diagram on paper and upload/attach separately as a pdf file to lab folder. An ER diagram (Entity-Relationship diagram) is a visual representation of the data model for a system. It shows the entities (things you want to store data about), their attributes (fields/columns), and the relationships between those entities.

Use the markdown cell below to document your design before implementing it in SQL.


### Task 2.1 Your Design

Identify the key functional dependencies in the raw dataset by listing which attributes determine others (e.g., student email determines student name and city, course code determines course title, etc.) and briefly justify each dependency based on real-world logic. Using these dependencies, they should then propose a normalized database schema by defining several tables (typically Students, Instructors, Courses, and Enrollments), and for each table specify the columns with data types, the primary key, any foreign keys and their references, and a short explanation of how the table removes redundancy or prevents update, insert, and delete anomalies. The goal is to produce a clear, well-reasoned 3NF design that reflects the entities and relationships present in the messy source data.

- **Functional dependencies:** 
  - The student email determines student name and city. In a real world context, this makes sense since a person would have the same email, name, and city across multiple purchases.
  - Instructor name determines instructor email. This makes sense in the real world, since the email address of an instructor depends on the instructor's name.
  - Course title dictates course category and course code. This makes sense, since the title and subject matter of the course dictate which category it belongs to. Additionally, each course has a corresponding course code.
  - Price is dependent on the course category and discount code. This makes sense since the amount the user spends on a course depends on the course that's being bought as well as whether or not they use a discount.
  - 
  - 
- **Proposed tables:**
  - Student
       - StudentID (Primary Key)
       - StudentEmail
       - Student_City
       - Student_Name
  - Instructor
       - InstructorID (Primary Key)  
       - InstructorEmail 
       - InstructorName
  - Courses
       - CourseID (Primary key)
       - InstructorID (Foreign Key)
       - CourseTitle
       - CourseCode
       - CourseCategory
- Enrollments
       - OrderID (Primary key)     
       - CourseID (Foreign key)
       - StudentID (Foreign Key)
       - Order Date
       - PaymentMethod
       - DiscountCode
       - EnrollmentStatus
       - Price 
  


For columns such as course, instructor, and student, there's only one primary key. As such, these tables automatically follow 3NF. However, for the enrollments table, I had to create a sort of composite primary key with OrderID, CourseID, and StudentID. This prevents 3NF since the circumstances for a student buying a course may be different depending on whether a discount method was used as well as the chosen payment method.

## Part 3 – Implement the Schema in SQLite (20 pts)

Now translate your design into **CREATE TABLE** statements.

1. Drop tables if they already exist (for repeatable runs).
2. Create core tables: `Students`, `Instructors`, `Courses`, `Orders/Enrollments`, etc.
3. Include:
   - `PRIMARY KEY`
   - `FOREIGN KEY` constraints
   - Reasonable data types

Complete the code below.

In [46]:
# update the schema to create all tables
schema_sql = """
DROP TABLE IF EXISTS Enrollments;
DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Students;
DROP TABLE IF EXISTS Instructors;

 CREATE TABLE Students (
     StudentID INTEGER PRIMARY KEY AUTOINCREMENT,     
     StudentEmail TEXT NOT NULL,
     StudentCity TEXT NOT NULL,
     StudentName TEXT NOT NULL
);

 CREATE TABLE Instructors (
     InstructorID INTEGER PRIMARY KEY AUTOINCREMENT,
     InstructorEmail TEXT NOT NULL,
     InstructorName TEXT NOT NULL
);

 CREATE TABLE Courses (
     CourseID INTEGER PRIMARY KEY AUTOINCREMENT,
     CourseCode TEXT NOT NULL,
     CourseTitle TEXT NOT NULL,
     CourseCategory TEXT NOT NULL,
     InstructorID INT NOT NULL,
     FOREIGN KEY (InstructorID) REFERENCES Instructors(InstructorID)
     );

 CREATE TABLE Enrollments (
     OrderID TEXT PRIMARY KEY,
     CourseID INT NOT NULL,
     StudentID INT NOT NULL,
     OrderDate TEXT NOT NULL,
     PaymentMethod TEXT NOT NULL,
     DiscountCode TEXT,
     EnrollmentStatus TEXT NOT NULL,
     price INTEGER NOT NULL,
     FOREIGN KEY (CourseID) REFERENCES Courses(CourseID),
     FOREIGN KEY (StudentID) REFERENCES Students(StudentID)
     );
"""
commands = schema_sql.split(';')
for x in commands:
    run_sql(x, show=False)
    


print("Executed schema_sql.")

Executed schema_sql.


## Part 4 – Staging Table & Data Loading (20 pts)
Loading the raw file into one staging table before populating the normalized tables is better than loading each table directly from the CSV because it creates a clean, consistent, and controllable pipeline for data transformation.
First, create a staging table `RawEnrollments` matching the CSV columns, then load data from the CSV into that table.

We will use **pandas** to load the cleaned CSV and then write it to SQL.


### Task 4.1
Create the staging table RawEnrollments and load the data from CSV

In [84]:

create_raw_sql = """
DROP TABLE IF EXISTS RawEnrollments;

CREATE TABLE RawEnrollments (
     OrderID INT,
     OrderDate TEXT,
     StudentName TEXT,
     StudentEmail TEXT,
     StudentCity TEXT,
     CourseCode TEXT,
     CourseTitle TEXT,
     CourseCategory TEXT,
     InstructorName TEXT,
     InstructorEmail TEXT,
     Price INT,
     PaymentMethod TEXT,
     DiscountCode TEXT,
     EnrollmentStatus TEXT
);
"""

commands = schema_sql.split(';')
for x in commands:
    run_sql(x, show=False)
print("RawEnrollments table created.")

df_raw = pd.read_csv("/srv/shared/data/EduTrack_Enrollments_Raw.csv")
df_raw.to_sql("RawEnrollments", conn, if_exists="replace", index=False)
print("Loaded data into RawEnrollments.")

run_sql("SELECT * FROM RawEnrollments LIMIT 5;")

RawEnrollments table created.
Loaded data into RawEnrollments.


,OrderID,OrderDate,StudentName,StudentEmail,StudentCity,CourseCode,CourseTitle,CourseCategory,InstructorName,InstructorEmail,Price,PaymentMethod,DiscountCode,EnrollmentStatus
0,1001,2024-09-03,DANIEL LEE,daniel.lee@example.com,San Francisco,DS101,Intro to Data Science,data science,Dr. Emily Zhang,emily.zhang@edutrack.com,FREE,None,SUMMER10,Completed
1,1001,2024-09-01,Bob Smith,bob.smith@example.com,new york,SQL201,Relational Databases for Analysts,Databases,Prof. Raj Patel,raj.patel@edutrack.com,FREE,Credit Card,SUMMER10,Actve
2,1002,2024-09-03,Daniel Lee,daniel.lee@example.com,San Francisco,DS101,intro to data science,Data Science,Dr. Emily Zhang,emily.zhang@edutrack.com,99.0,PayPal,None,active
3,1003,2024-09-01,Bob Smith,bob.smith@example.com,New York,SQL201,RELATIONAL DATABASES FOR ANALYSTS,Databases,Prof. Raj Patel,raj.patel@edutrack.com,79,Credit Card,SUMMER10,Refunded
4,1004,2024-09-03,Daniel Lee,daniel.lee@example.com,San Francisco,DS101,Intro to Data Science,Data Science,Dr. Emily Zhang,emily.zhang@edutrack.com,99.0,Credit Card,SUMMER10,Actve


### Task 4.2 Populate Normalized Tables from `RawEnrollments`

Now write `INSERT INTO ... SELECT ...` statements to populate your normalized tables.

Examples you might need (adapt to your schema):
- Insert distinct students
- Insert distinct instructors
- Insert distinct courses
- Insert enrollment/order records with correct foreign keys


In [57]:
load_normalized_sql = """

INSERT INTO Students (StudentEmail, StudentCity, StudentName)
SELECT DISTINCT StudentEmail, StudentCity, StudentName
FROM RawEnrollments;

INSERT INTO Instructors (InstructorEmail, InstructorName)
SELECT DISTINCT InstructorEmail, InstructorName
FROM RawEnrollments
WHERE InstructorEmail IS NOT NULL;

INSERT INTO Courses (InstructorID, CourseCode, CourseTitle, CourseCategory)
SELECT DISTINCT Instructors.InstructorID, RawEnrollments.CourseCode, RawEnrollments.CourseTitle, RawEnrollments.CourseCategory
FROM RawEnrollments
JOIN Instructors ON 
RawEnrollments.InstructorName = Instructors.InstructorName;


INSERT INTO Enrollments (CourseID, StudentID, OrderDate, PaymentMethod, DiscountCode, EnrollmentStatus, Price)
SELECT DISTINCT Courses.CourseID, Students.StudentID, RawEnrollments.OrderDate, RawEnrollments.PaymentMethod, RawEnrollments.DiscountCode, RawEnrollments.EnrollmentStatus, RawEnrollments.Price
FROM RawEnrollments
JOIN Courses
ON Courses.CourseCode = RawEnrollments.CourseCode AND Courses.CourseTitle = RawEnrollments.CourseTitle AND Courses.CourseCategory = RawEnrollments.CourseCategory
JOIN Students
ON
Students.StudentName = RawEnrollments.StudentName AND Students.StudentEmail = RawEnrollments.StudentEmail AND Students.StudentCity = RawEnrollments.StudentCity
WHERE
RawEnrollments.PaymentMethod IS NOT NULL;

"""

commands = load_normalized_sql.split(';')
for x in commands:
    run_sql(x, show=False)

print("Executed load_normalized_sql.")


Executed load_normalized_sql.


## Part 5 – UPDATE, DELETE, and Analytical Queries (15 pts)

Fill in the cells below with SQL for the tasks described in the assignment:

1. Update a record to `Refunded`.
2. Standardize `student_city` to `New York City` for New York students.
3. Update the standard price for `DS101` in the `Courses` table (not historical orders).
4. Delete refunded orders from the tables.
5. Analytical queries (counts, revenue, joins, etc.). Answer your own questions. For example, how many students are registered for DS101? What is the total revenue generated from the course? What is the total enrollment in DS101 etc.


In [58]:
update_sql = """
UPDATE Enrollments SET EnrollmentStatus = 'Refunded' WHERE OrderID = 1005;
UPDATE Students SET StudentCity = 'New York City' WHERE StudentCity = 'New York';
UPDATE Students SET StudentCity = 'New York City' WHERE StudentCity = 'New York';
UPDATE RawEnrollements JOIN Courses ON Courses.CourseCode = RawEnrollments.CourseCode AND Courses.CourseTitle = RawEnrollments.CourseTitle AND Courses.CourseCategory = RawEnrollments.CourseCategory SET RawEnrollments.Price = 1000 WHERE Courses.CourseCode = 'DS101';
"""
commands = load_normalized_sql.split(';')
for x in commands:
    run_sql(x, show=False)

print("Executed update_sql")

Executed update_sql


In [71]:
delete_sql = """
DELETE FROM Courses WHERE (CourseCode, CourseTitle, CourseCategory) IN (SELECT CourseCode, CourseTitle, CourseCategory FROM RawEnrollments WHERE EnrollmentStatus = 'Refunded');
DELETE FROM Students WHERE (StudentEmail, StudentCity, StudentName) IN (SELECT StudentEmail, StudentCity, StudentName FROM RawEnrollments WHERE EnrollmentStatus = 'Refunded');
DELETE FROM Instructors WHERE (InstructorEmail, InstructorName) IN (SELECT InstructorEmail, InstructorName FROM RawEnrollments WHERE EnrollmentStatus = 'Refunded');
DELETE FROM Enrollments WHERE EnrollmentStatus = 'Refunded';
"""
commands = load_normalized_sql.split(';')
for x in commands:
    run_sql(x, show=False)
print("Executed delete_sql")

Executed delete_sql


In [79]:
analysis_sql = """
SELECT Courses.CourseTitle, SUM(RawEnrollments.Price) FROM RawEnrollments JOIN Courses ON Courses.CourseCode = RawEnrollments.CourseCode AND Courses.CourseTitle = RawEnrollments.CourseTitle AND Courses.CourseCategory = RawEnrollments.CourseCategory GROUP BY Courses.CourseTitle ORDER BY SUM(RawEnrollments.Price) DESC;
"""
run_sql(analysis_sql)
print("What is the most profitable course? Note: Might have to add values together to account for different formatting in raw data.")

,CourseTitle,SUM(RawEnrollments.Price)
0,Intro to Data Science,404114.0
1,Intro to Machine Learning,243552.0
2,Relational Databases for Analysts,228158.0
3,intro to data science,120099.0
4,Intro to Data Science,43430.0
5,INTRO TO DATA SCIENCE,35131.0
6,RELATIONAL DATABASES FOR ANALYSTS,20382.0
7,intro to machine learning,10750.0
8,Relational Databases for Analysts,10707.0
9,relational databases for analysts,10191.0


What is the most profitable course? Note: Might have to add values together to account for different formatting in raw data.


## Part 6 – Reflection on Normalization (10 pts)

Use this markdown cell (or an external document) to reflect on:

- Which normal forms (1NF, 2NF, 3NF) your design satisfies
- Examples of how your design avoids update/insert/delete anomalies
- How you would extend the schema for multiple instructors per course, multiple discounts per order, etc.

You may use cGPT to find answers to these questions.


In [ ]:
My design satisfies up to 3NF. I know this because for all tables, all of the columns have atomic values. Additionally, since most tables have one primary key. Even with the enrollments table, all of the values depend on either a primary key or a foreign key. My design avoids any anomalies since it splits up the raw data into the most essential parts so only the relevant columns are accessed when needed. When thinking about multiple instructors or multiple discounts, it may be best to create junction tables to account for the many to many relationships between each column while keeping 3NF.

### Submission Instructions
Rename the lab as netID_Lab04.ipynb and submit to codebench. (eg: adg133_Lab04.ipynb)